# Module 5: Clustering Concepts

## What to learn in this module

In this module, we will learn community detection, one of the most widely-used yet controversial techniques in network analysis. We will learn:
- What is community structure in networks?
- How to operationalize community structure?
- How to find communities in networks?
- Limitations of community detection
- **Keywords**: community detection, assortativity, modularity, resolution limit, rugged landscape, random graph, label switching algorithm, Louvain algorithm, stochastic block model, the configuration model.

## Understanding communities in networks

Birds of a feather flock together, and so do many other things. For instance, we have a group of friends with similar interests who hang out together frequently but may not interact as much with other groups.

![Birds of a feather](https://t4.ftcdn.net/jpg/08/10/89/17/360_F_810891701_xy4NsqgdqllMfKDfV6V27ycrw8FLFqrw.jpg)

In networks, communities are groups of nodes that share similar connection patterns. These communities do not always mean densely-connected nodes. Sometimes, a community can be nodes that are not connected to each other, but connect similarly to other groups. For instance, in a user-movie rating network, a community might be users with similar movie tastes, even if they don't directly connect to each other.

![Community structure in a social network](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Network_Community_Structure.svg/220px-Network_Community_Structure.svg.png)

Communities reflect underlying mechanisms of network formation and underpin the dynamics of information propagation. Examples include:

1. Homophily: The tendency of similar nodes to form connections.
2. Functional groups: Nodes that collaborate for specific purposes.
3. Hierarchical structure: Smaller communities existing within larger ones.
4. Information flow: The patterns of information, influence, or disease propagation through the network.

This is why network scientists are sooo obsessed with community structure in networks. See {footcite}`fortunato2010community,fortunato2016community,peixoto2019bayesian` for comprehensive reviews on network communities.

## Pattern matching approach

Community detection is an abstract unsupervised problem. It is abstract because there is no clear-cut definition or ground truth to compare against. The concept of a community in a network is subjective and highly context-dependent.

A classical approach to community detection is based on *pattern matching*. Namely, we first explicitly define a community by a specific connectivity pattern of its members. Then, we search for these communities in the network.

::: {#fig-clique}

<img src="https://pythonhosted.org/trustedanalytics/R_images/k-clique_201508281155.png" alt="Clique graph" width="80%">

Cliques of different sizes. Taken from [https://pythonhosted.org/trustedanalytics/python_api/graphs/graph-/kclique_percolation.html](https://pythonhosted.org/trustedanalytics/python_api/graphs/graph-/kclique_percolation.html)
:::

Perhaps, the strictest definition of a community is a *clique*: a group of nodes all connected to each other. Examples include triangles (3-node cliques) and fully-connected squares (4-node cliques). However, cliques are often too rigid for real-world networks. In social networks, for instance, large groups of friends rarely have every member connected to every other, yet we want to accept such "in-perfect" social circles as communities. This leads to the idea of relaxed versions of cliques, called **pseudo-cliques**.

Pseudo-cliques are defined by relaxing at least one of the following three dimensions of strictness:

1. Degree: Not all nodes need to connect to every other node.
   - **$k$-plex**: each node connects to all but $k$ others in the group {footcite}`seidman1978graph`.
   - **$k$-core**: each node connects to $k$ others in the group {footcite}`seidman1983network`.
2. Density: The overall connection density can be lower.
   - **$\rho$-dense subgraphs**, with a minimum edge density of $\rho$ {footcite}`goldberg1984finding`.
3. Distance: Nodes can be further apart.
   - **$n$-clique**, where all nodes are within n steps of each other {footcite}`luce1950connectivity`.
4. Combination of the above:
   - **n-clan** and **n-club** {footcite}`mokken1979cliques`
   - **$k$-truss**, a maximal subgraph where all edges participate in at least $k-2$ triangles {footcite}`saito2008extracting,cohen2009graph,wang2010triangulation`.
   - **$\rho$-dense core**, a subgraph with minimum conductance $\rho$ {footcite}`koujaku2016dense`.

::: {#fig-clique-pattern}

<img src="https://ars.els-cdn.com/content/image/1-s2.0-S0378873315000520-gr1.jpg" alt="Pseudo-clique patterns" width="80%" style="display: block; margin-left: auto; margin-right: auto;">

Illustation of different pseudo cliques. Taken from {footcite}`koujaku2016dense`.

:::

## Graph cut optimization approach

Another approach from computer science is to treat a community detection problem as an *optimization* problem. An early example is the **graph cut** problem, which asks to find the minimum number of edges to cut the graph into two disconnected components.

Specifically, let us consider cutting the network into two communities. Let $V_1$ and $V_2$ be the set of nodes in the two communities. Then, the cut is the number of edges between the two communities, which is given by

$$
\begin{align}
\text{Cut}(V_1, V_2) = \sum_{i \in V_1} \sum_{j \in V_2} A_{ij}
\end{align}
$$

Now, the community detection problem is translated into **an optimization problem**, with the goal of finding a cut $V_1, V_2$ that minimizes $\text{Cut}(V_1, V_2)$.

The description of this problem is not complete 😈. Let's find out what is missing by playing with the optimization problem.

::: {.callout-note title="Exercise"}
:class: tip

Can you identify what is missing in the description of the graph cut problem? Without this, the best cut is trivial. {{ "<a href='BASE_URL/vis/community-detection/index.html?scoreType=graphcut&numCommunities=2&randomness=1&dataFile=two-cliques.json'>Graph Cut Problem 🎮</a>".replace('BASE_URL', base_url) }}

::: {.callout collapse="true"}
**Click to reveal the answer!**

The missing element is a constraint: each community must contain at least one node. Without this, the trivial solution of placing all nodes in a single community would always yield a cut of zero.
:::

:::

## Modularity: measuring assortativity against null models

**Modularity** is by far the most widely used method for community detection. Modularity can be derived in many ways, but we will follow the one derived from assortativity.

**Assortativity** is a measure of the tendency of nodes to connect with nodes of the same attribute. The attribute, in our case, is the community that the node belongs to, and we say that a network is assortative if nodes of the same community are more likely to connect with each other than nodes of different communities.

Let's think about assortativity by using color balls and strings! 🎨🧵

Imagine we're playing a game as follows:

1. Picture each connection in our network as two colored balls joined by a piece of string. 🔴🟢--🔵🟡
2. The color of each ball shows which community it belongs to.
3. Now, let's toss all these ball-and-string pairs into a big bag.
4. We'll keep pulling out strings with replacement and checking if the balls on each end match colors.

The more color matches we find, the more assortative our network is. But, there's a catch! What if we got lots of matches just by luck? For example, if all our balls were the same color, we'd always get a match. But that doesn't tell us much about our communities. So, to be extra clever, we compare our results to a "random" version (null model):

1. We snip all the strings and mix up all the balls.
2. Then we draw pairs of balls at random *with replacement* and see how often the colors match.

By comparing our original network to this mixed-up version, we can see if our communities are really sticking together more than we'd expect by chance. This comparison against the random version is the heart of modularity. Unlike graph cut methods that aim to maximize assortativity directly, modularity measures assortativity *relative* to *a null model*.

::: {#fig-fig-modularity-game}

<img src="../figs/modularity.jpg" alt="Single node failure" width="100%">

Illustration of how modularity measures assortativity relative to a null model.
:::

Now, let's put on our math hats and make this colorful game a bit more precise. Let's introduce some helpful symbols to describe our network:
- $N$: This is our total number of nodes (or balls in our game)
- $M$: The number of edges (or strings) connecting our nodes
- $A_{ij}$: Adjacency matrix. If $A_{ij} = 1$, it means node $i$ and node $j$ are connected. If $A_{ij} = 0$, they're not connected.
- $k_i$: Degree of node $i$, i.e., how many edges a node has.
- $c_i$: Community of node $i$, i.e., which community a node belongs to.
- $\delta(c_i, c_j)$: Kronecker delta function. It gives us 1 if nodes $i$ and $j$ are the same color, and 0 if they're different.

::: {.callout-note title="Exercise"}
:class: tip

What is the probability of color matches for a given network? Derive the probability by using $\sum, M, A_{ij}, \delta(c_i, c_j)$.

::: {.callout-note collapse="true" title="Hint"}
Let's think about our colorful bag of balls and strings! 🎨🧵
First, ask yourself:
1. How many strings do we have in total? (This is our M!)
2. Now, out of all these strings, how many are the same color on both ends?
:::

:::

::: {.callout-note title="Exercise"}
:class: tip

What is the probability of color matches for the random version? Derive the probability by using $\sum, M, \delta(c_i, c_j), k_i,k_j$.

::: {.callout collapse="true"}
**Hint**
1. Imagine a big bag full of colorful balls, but this time without any strings. 🔴🟢🔵🟡
2. Now, think about picking one ball out of the bag. What are the chances of picking a specific color?
3. Then, put that ball back and pick another one. What are the odds this second ball matches the color of the first one?
:::

:::

The full modularity formula is covered in the coding section 😉.

## Pen and Paper Exercise

✍️ [Pen and Paper Exercise](./pen-and-paper/exercise.pdf) 🚢

```{footbibliography}
```